In [ ]:
import dynamoon as dm
import matplotlib.pyplot as plt

In [ ]:
star = dm.star(1, 1, [0.5, 0.5])
planet = dm.rock(10, 1)
moon = dm.rock(0.5, 0.5)

In [ ]:
t = np.linspace(3.5, 4.5, 100000)
sys = dm.system(star, planet, moon)
sys.set_planet_orbit(t0=4, e=0.0, P=365, Omega=180, w=0, i=90)
sys.set_moon_orbit(t0=0.15, e=0.1, P=28, Omega=80, w=0, i=90)

In [ ]:
%time flux = sys.flux(t)
plt.plot(t, flux, '-')
#plt.xlim(4.26, 4.28)
#plt.axvline(3.9755)

In [ ]:
fig = plt.figure(figsize=(5, 5))
sys.draw_config(fig.gca(), 4.27)
plt.xlim(-1.1, -0.9)
plt.ylim(-0.1, 0.1)

In [ ]:
from IPython.display import HTML

stkwargs = {"fill": True, "color":"#ffbb4d"}
plkwargs = {"fill": True, "color":"#4c89d4", "alpha":1}
mokwargs = {"fill": True, "color":"#ff4a4a", "alpha":1}

animation = sys.animate(np.linspace(3.6, 4.4, 200), 
                        stkwargs=stkwargs, 
                        plkwargs=plkwargs, 
                        mokwargs=mokwargs, duration=5)

HTML(animation.to_html5_video())

In [ ]:
import batman
from astropy import constants as ac

params = batman.TransitParams()
params.t0 = 4                       
params.per = 100                      
params.rp = 1 / sys.star.radius                    
params.a = sys.starplanet.a * ac.au.value / (sys.star.radius * ac.R_earth.value)
params.inc = 90                     
params.ecc = 0.2                     
params.w = 180                       
params.u = [0.5, 0.5]                
params.limb_dark = "quadratic"       

In [ ]:
%time m = batman.TransitModel(params, t)    
%time flux_batman = m.light_curve(params)          

In [ ]:
plt.plot(t, flux_batman, color='k')
plt.plot(t, flux+1)
plt.xlim(3, 5)

In [ ]:
plt.plot(t, ((flux+1) - flux_batman)*1e6)
plt.xlim(3, 5)
plt.ylim(-2, 1)

In [ ]:
plt.plot(flux_batman)

In [ ]:
0.003216 / 0.005202

In [ ]:
bb = 0.0
Tb = 13
ep = 0.01
P = 365 * 24 
i = 90 * np.pi / 180
np.sqrt(1 - bb**2) / np.sin(Tb * np.pi / P * np.sqrt(1 - ep**2)) / np.sin(i)

In [ ]:
ac.au.value / ac.R_sun.value

In [ ]:
import dynamoon as dm
import matplotlib.pyplot as plt

taub = 4
rp = ac.R_earth.value / ac.R_sun.value
Pb = 365
bb = 0.0
mm = 0.012
rm = 0.25 * rp
bm = 0.1
emcosw = 0.0
emsinw = 0.0
omegam = 0.0
u1 = 0.5
u2 = 0.5
Pm = 28
%time sys = dm.system.from_fitting_params(taub, rp, Pb, bb, mm, rm, bm, Pm, emcosw, emsinw, omegam, u1, u2)

In [ ]:
t = np.linspace(2, 5, 100000)
%time flux = sys.flux(t)
plt.plot(t*24, flux, '-')
plt.axvline(taub*24 - 13/2)
plt.axvline(taub*24 + 13/2)

In [ ]:
from IPython.display import HTML

stkwargs = {"fill": True, "color":"#ffbb4d"}
plkwargs = {"fill": True, "color":"#4c89d4", "alpha":1}
mokwargs = {"fill": True, "color":"#ff4a4a", "alpha":1}

animation = sys.animate(np.linspace(, 200), 
                        stkwargs=stkwargs, 
                        plkwargs=plkwargs, 
                        mokwargs=mokwargs, duration=5)

HTML(animation.to_html5_video())

In [ ]:
import emcee

In [6]:
noise = np.loadtxt('/Users/tgordon/research/nirspec/JexoSim/output/case_studies/ngts11b_1.dat')

In [7]:
from astropy.io import fits
file = '/Users/tgordon/research/nirspec/JexoSim/output/jexosim_intermediate_NIRSpec_BOTS_PRISM_NGTS-11 b_2021_03_23_1714_15.fits'
file = open(file, 'rb')
data = fits.open(file)
wv = data[2].data
wv = np.array([x[0] for x in np.array(wv)])
t = data[3].data
t = np.array([x[0] for x in np.array(t)])

In [8]:
noise = np.sum(noise, axis=1)

In [9]:
tp = t / (60 * 60 * 24)
plt.plot(tp, noise/np.mean(noise))

NameError: name 'plt' is not defined

In [ ]:
import imp
imp.reload(dm)
taub = 0.2
rp = 5
Pb = 35
bb = 0.0
mm = 0.012 
rm = 0.4 * rp
bm = 0.1
emcosw = 0.0
emsinw = 0.0
omegam = 0.0
u1 = 0.5
u2 = 0.5
Pm = 28
%time sys = dm.system.from_fitting_params(taub, rp, Pb, bb, mm, rm, bm, Pm, emcosw, emsinw, omegam, u1, u2)

In [ ]:
%time flux = sys.flux(tp)
plt.plot(tp, flux)

In [ ]:
plt.plot(tp, flux + noise/np.mean(noise), '.')

In [ ]:
import celerite2

y = flux + noise/np.mean(noise)
x = tp

def logp(args, x, y):
    
    ls0, lw0, taub, rp, Pb, bb, mm, rm, bm, emcosw, emsinw, omegam, u1, u2, Pm, yerr, f0 = args
    s0, w0 = np.exp(ls0), np.exp(lw0)
    
    bounds = {"ls0": (-np.inf, 0),
              "lw0": (-np.inf, np.inf),
              "taub": (x.min(), x.max()), 
              "rp": (0, np.inf), 
              "Pb": (0, np.inf), 
              "bb": (0, 1), 
              "mm": (0, np.inf), 
              "rm": (0, np.inf), 
              "bm": (0, 1), 
              "emcosw": (0, 1), 
              "emsinw": (0, 1), 
              "omegam": (0, 360), 
              "u1": (0, 1), 
              "u2": (0, 1), 
              "Pm": (0, np.inf),
              "yerr": (0, 1), 
              "f0": (-np.inf, np.inf)}
    
    inbound = np.array([(a <= b[1]) & (a >= b[0]) for a, b in zip(args, list(bounds.values()))])
    if np.any(inbound) == False:
        return -np.inf
    
    if not ((u1 + u2) < 1) & (u1 > 0) & ((u1 + 2*u2) > 0):
        return -np.inf
    
    
    term = celerite2.terms.SHOTerm(S0=s0, w0=w0, Q=1/np.sqrt(2))
    sys = dm.system.from_fitting_params(taub, rp, Pb, bb, mm, rm, bm, Pm, emcosw, emsinw, omegam, u1, u2)
    mu = sys.flux(x)
    r = y - mu - f0
    gp = celerite2.GaussianProcess(term, mean=0)
    gp.compute(x, yerr = yerr*np.ones_like(x))
    return gp.log_likelihood(r)

nll = lambda args, x, y: -logp(args, x, y)

In [ ]:
p0 = {"ls0": -23,
      "lw0": 6,
      "taub": 0.2,
      "rp": 5,
      "Pb": 35,
      "bb": 0.0,
      "mm": 0.012,
      "rm": 2,
      "bm": 0.1,
      "emcosw": 0.0,
      "emsinw": 0.0,
      "omegam": 0.0,
      "u1": 0.5,
      "u2": 0.3,
      "Pm": 28,
      "yerr": 0.002,
      "f0": 1}

nll(list(p0.values()), x, y)

In [ ]:
from scipy.optimize import minimize

res = minimize(nll, list(p0.values()), args=(x, y))

In [ ]:
ls0, lw0, taub, rp, Pb, bb, mm, rm, bm, emcosw, emsinw, omegam, u1, u2, Pm, yerr, f0 = res.x
s0, w0 = np.exp(ls0), np.exp(lw0)
term = celerite2.terms.SHOTerm(S0=s0, w0=w0, Q=1/np.sqrt(2))
sys = dm.system.from_fitting_params(taub, rp, Pb, bb, mm, rm, bm, Pm, emcosw, emsinw, omegam, u1, u2)
gp = celerite2.GaussianProcess(term, mean=0)
gp.compute(x, yerr = yerr*np.ones_like(x))
plt.plot(x, sys.flux(x) + gp.dot_tril(np.random.randn(len(x))))
plt.plot(x, sys.flux(x), color='k')

In [ ]:
import emcee
nparams = len(p0)
nwalkers = 36
x0 = np.array(res.x)
x0 = np.random.rand(nwalkers, nparams)*1e-8 + x0

filename = "../mcmc_output/1transit.h5"
backend = emcee.backends.HDFBackend(filename)
backend.reset(nwalkers, nparams)

sampler = emcee.EnsembleSampler(nwalkers, nparams, logp, args=[x, y], backend=backend)

In [ ]:
sampler.run_mcmc(x0, 1000, progress=True)

In [ ]:
import corner
trace = emcee.backends.HDFBackend(filename)
samples = trace.get_chain(flat=True)
fig = corner.corner(samples, labels=list(p0.keys()));